In [ ]:
import os
import PySpin
import cv2
import time
import numpy as np

# Retrieve singleton reference to system object
system = PySpin.System.GetInstance()

# Get current library version
version = system.GetLibraryVersion()
print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))


# Retrieve list of cameras from the system
cam_list = system.GetCameras()
num_cameras = cam_list.GetSize()

print('Number of cameras detected: %d' % num_cameras)

cam = cam_list.GetByIndex(0)
cam.Init()
cam.PixelFormat.SetValue(PySpin.PixelFormat_BayerGB8)
print('Pixel format:', cam.PixelFormat.GetCurrentEntry().GetSymbolic())
device_model = cam.DeviceModelName.ToString()
print('Device model:', device_model)

In [ ]:
# image acquisition
cam.AcquisitionMode.SetValue(PySpin.AcquisitionMode_Continuous)
# # trigger setting
cam.TriggerMode.SetValue(PySpin.TriggerMode_Off)
cam.TriggerSource.SetValue(PySpin.TriggerSource_Software)
cam.TriggerMode.SetValue(PySpin.TriggerMode_On)
print("set trigger mode software")
# Get serial number
print('Serial number:', cam.DeviceSerialNumber.ToString())
cam.BeginAcquisition()

print('start capturing')
elps = []
while True:
    start = time.time()
    cam.TriggerSoftware()
    image_result = cam.GetNextImage()
    if image_result.IsIncomplete():
        print('Error')
#     image_converted = image_result.Convert(PySpin.PixelFormat_BGR8, PySpin.HQ_LINEAR) # slow
#     img = image_converted.GetNDArray()
    img = image_result.GetNDArray()
    img = cv2.cvtColor(img, cv2.COLOR_BAYER_GR2BGR) # bayer pattern is correct?
    img = cv2.resize(img, (640, 640))

    cv2.imshow('color', img)
    key= cv2.waitKey(5)
    elps.append((time.time()-start))

    if key==27:
        break
    image_result.Release()

cv2.destroyAllWindows()
print('finish capturing')
print('Average fps:', 1/np.array(elps).mean())

cam.EndAcquisition()
cam.DeInit()
del cam
# Clear camera list before releasing system
cam_list.Clear()

# Release system instance
system.ReleaseInstance()
del system
print('release everything')